In [ ]:
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from sklearn.preprocessing import StandardScaler

In [267]:
def synthesize_cloud_csv(num_rows = 1000):
    rows = []

    for _ in range(num_rows):

        profile = np.random.choice(["normal", "automated", "attacker"], p = [0.7, 0.15, 0.15])

        hour = np.random.randint(0, 24)
        day = np.random.randint(0, 7)

        if profile == "normal":
            region = np.random.choice([0, 1], p=[0.8, 0.2])
            ip_risk = np.random.beta(2, 10)
            req_5m = np.random.poisson(2)
            req_1h = np.random.poisson(12)
            failed_ratio = np.random.beta(1, 15)
            key_sens = np.random.uniform(0.2, 0.6)
            hist_access = np.random.beta(5, 2)
            principal = np.random.choice([0, 1], p=[0.8, 0.2])
            fraud = 0

        elif profile == "automated":
            region = np.random.choice([0, 2])
            ip_risk = np.random.beta(2, 4)
            req_5m = np.random.poisson(8)
            req_1h = np.random.poisson(60)
            failed_ratio = np.random.beta(1, 5)
            key_sens = np.random.uniform(0.4, 0.7)
            hist_access = np.random.beta(6, 1)
            principal = 2
            fraud = 0

        else:
            region = np.random.choice([3, 4])
            ip_risk = np.random.beta(8, 1)
            req_5m = np.random.poisson(20)
            req_1h = np.random.poisson(120)
            failed_ratio = np.random.beta(5, 1)
            key_sens = np.random.uniform(0.7, 1.0)
            hist_access = np.random.beta(1, 6)
            principal = np.random.choice([0, 1])
            hour = np.random.choice([0,1,2,3,4,22,23])
            fraud = 1

        identity = 1 if region >= 3 else 0
        unusual = 1 if hour in [0,1,2,3,4,22,23] else 0
        region_risk = 1 if region >= 3 else 0
        freq = 1 if req_5m > 10 else 0

        rows.append([
            hour, 
            day, 
            region, 
            ip_risk, 
            req_5m,
            req_1h, 
            principal,
            key_sens, 
            hist_access, 
            failed_ratio,
            identity, 
            unusual, 
            region_risk, 
            freq, 
            fraud
        ])

    return pd.DataFrame(rows, columns=[
        "hour_of_day",
        "day_of_week",
        "region_code",
        "source_ip_risk",
        "request_count_5m",
        "request_count_1h",
        "principal_type",
        "key_sensitivity",
        "historical_access_rate",
        "failed_auth_ratio",
        "identity_mismatch",
        "unusual_time",
        "region_risk",
        "request_frequency",
        "possibility_of_fraud"
    ])

In [ ]:
def build_security_enrichment_model(input_dim):

    inputs = Input(shape = (input_dim,))
    x = layers.BatchNormalization()(inputs)

    x = layers.Dense(128, activation = "relu")(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(64, activation = "relu")(x)
    shared = layers.Dense(64, activation="relu")(x)

    def head(name):
        return layers.Dense(1, activation = "sigmoid", name=name)(
            layers.Dense(32, activation = "relu")(shared)
        )

    identity = head("identity_mismatch")
    unusual = head("unusual_time")
    region = head("region_risk")
    freq = head("request_frequency")

    combined = layers.Concatenate()([identity, unusual, region, freq, shared])
    fraud = layers.Dense(1, activation = "sigmoid", name="possibility_of_fraud")(
        layers.Dense(32, activation = "relu")(combined)
    )

    model = models.Model(inputs, [identity, unusual, region, freq, fraud])

    model.compile(
        optimizer = tf.keras.optimizers.Adam(0.0003),
        loss = "binary_crossentropy",
        metrics = ["accuracy"]
    )

    return model

In [269]:
def train_model(model, X, y):
    model.fit(X, y, epochs = 12, batch_size = 64, validation_split = 0.2, verbose = 1)

In [270]:
def enrich_events(df, model):

    X = df.iloc[:, :10].values
    preds = model.predict(X)

    enrichment = pd.DataFrame(np.hstack(preds), columns=[
        "identity_mismatch",
        "unusual_time",
        "region_risk",
        "request_frequency",
        "possibility_of_fraud"
    ])

    return pd.concat([df.iloc[:, :10].reset_index(drop = True), enrichment], axis = 1)

In [271]:
def run_pipeline():

    df = synthesize_cloud_csv(10000)

    X = df.iloc[:, :10].values

    y = [
        df["identity_mismatch"].values,
        df["unusual_time"].values,
        df["region_risk"].values,
        df["request_frequency"].values,
        df["possibility_of_fraud"].values,
    ]

    model = build_security_enrichment_model(X.shape[1])

    train_model(model, X, y)

    enriched = enrich_events(df, model)

    return enriched

In [272]:
if __name__ == "__main__":
    df = synthesize_cloud_csv(10000)
    X = df.iloc[:, :10].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    y = [
        df["identity_mismatch"].values,
        df["unusual_time"].values,
        df["region_risk"].values,
        df["request_frequency"].values,
        df["possibility_of_fraud"].values,
    ]

    model = build_security_enrichment_model(X_scaled.shape[1])
    train_model(model, X_scaled, y)

    enriched_events = enrich_events(df, model)
    print(enriched_events.head())
    enriched_events.to_csv("output_file.csv", index = False)

    # Model Export/Deployment
    model.save("kc_ids_model.keras")
    joblib.dump(scaler, "feature_scaler.joblib")

Epoch 1/12
125/125 [==============================] - 4s 6ms/step - loss: 1.7189 - identity_mismatch_loss: 0.2688 - unusual_time_loss: 0.5687 - region_risk_loss: 0.2551 - request_frequency_loss: 0.3303 - possibility_of_fraud_loss: 0.2960 - identity_mismatch_accuracy: 0.9534 - unusual_time_accuracy: 0.7285 - region_risk_accuracy: 0.9724 - request_frequency_accuracy: 0.9134 - possibility_of_fraud_accuracy: 0.9681 - val_loss: 0.6576 - val_identity_mismatch_loss: 0.0087 - val_unusual_time_loss: 0.4992 - val_region_risk_loss: 0.0071 - val_request_frequency_loss: 0.1281 - val_possibility_of_fraud_loss: 0.0146 - val_identity_mismatch_accuracy: 1.0000 - val_unusual_time_accuracy: 0.7405 - val_region_risk_accuracy: 1.0000 - val_request_frequency_accuracy: 0.9610 - val_possibility_of_fraud_accuracy: 1.0000
Epoch 2/12
125/125 [==============================] - 0s 4ms/step - loss: 0.5774 - identity_mismatch_loss: 0.0058 - unusual_time_loss: 0.4641 - region_risk_loss: 0.0052 - request_frequency_los